# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

Импорт всех необходимых библиотек

In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from scipy import stats as st
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Открытие наборов данных

In [2]:
df0 = pd.read_csv("/datasets/geo_data_0.csv")
df1 = pd.read_csv("/datasets/geo_data_1.csv")
df2 = pd.read_csv("/datasets/geo_data_2.csv")

Согласно информации по этим наборам пропусков нет и предобработка данных не понадобится, так как данные синтетические

In [3]:
df0.info()
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

Удаляю столбец с уникальным идентификатором каждого месторождения, т.к. этот признак помешает обучению

In [4]:
del df0["id"]
del df1["id"]
del df2["id"]

Так выглядят данные: три столбца признаков и целевой признак

In [5]:
df0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


#### Выводы
1. 3 набора данных по трем регионам добычи
2. В каждом наборе 10000 записей (месторождений)
3. В каждом наборе 3 признака и количество нефти, которую возможно добыть
4. Данные синтетические

# 2. Обучение и проверка модели

Подготавливаю данные для обучения. Данные будут разбиты на обучающие и валидационные в соотношении 75-25. Ниже данные для первого региона 

In [6]:
X0 = df0.drop('product', axis=1)
y0 = df0['product'] 

In [7]:
X0_train, X0_val, y0_train, y0_val = train_test_split(X0, y0, test_size=0.25, random_state=12345)

Для второго

In [8]:
X1 = df1.drop('product', axis=1)
y1 = df1['product'] 

In [9]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, test_size=0.25, random_state=12345)

И для третьего

In [10]:
X2 = df2.drop('product', axis=1)
y2 = df2['product'] 

In [11]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.25, random_state=12345)

Обучаю модель для регрессии по первому региону. Затем нахожу предикт. После чего нахожу среднее по предикту и RMSE

In [12]:
reg0 = LinearRegression().fit(X0_train, y0_train)

In [13]:
y0_pred = reg0.predict(X0_val)

In [14]:
y0_pred_mean = y0_pred.mean()

In [15]:
rmse0 = mean_squared_error(y0_val, y0_pred) ** 0.5

Аналогичные шаги для второго региона

In [16]:
reg1 = LinearRegression().fit(X1_train, y1_train)

In [17]:
y1_pred = reg1.predict(X1_val)

In [18]:
y1_pred_mean = y1_pred.mean()

In [19]:
rmse1 = mean_squared_error(y1_val, y1_pred) ** 0.5

И для третьего

In [20]:
reg2 = LinearRegression().fit(X2_train, y2_train)

In [21]:
y2_pred = reg2.predict(X2_val)

In [22]:
y2_pred_mean = y2_pred.mean()

In [23]:
rmse2 = mean_squared_error(y2_val, y2_pred) ** 0.5

In [24]:
print("Первый регион:")
print("Среднее значение по предикту:", y0_pred_mean)
print("RMSE:", rmse0)
print("Второй регион:")
print("Среднее значение по предикту:", y1_pred_mean)
print("RMSE:", rmse1)
print("Третий регион:")
print("Среднее значение по предикту:", y2_pred_mean)
print("RMSE:", rmse2)

Первый регион:
Среднее значение по предикту: 92.59256778438038
RMSE: 37.5794217150813
Второй регион:
Среднее значение по предикту: 68.728546895446
RMSE: 0.893099286775616
Третий регион:
Среднее значение по предикту: 94.96504596800489
RMSE: 40.02970873393434


#### Выводы
1. По первому и третьему региону неплохое среднее значение, но болшая среднеквадратичная ошибка
2. Второй регион показывает среднее значение хуже, но при этом RMSE очень низкое, что значит, что модель очень качественно предсказала значения 

# 3. Подготовка к расчёту прибыли

Объявляю константы, приведенные в описании проекта, а именно:
- Бюджет, заложенный в освоение 200 скважин
- Доход за тысячу бареллей
- Возможные расходы, допустимые для одной скважины
- Количество тысяч бареллей, необходимых для доходности

In [25]:
BUDGET = 10000000000
INCOME = 450000
COST_PER_REGION = BUDGET / 200
VOLUME = BUDGET / INCOME / 200

In [26]:
VOLUME

111.11111111111111

# 4. Расчёт прибыли и рисков 

Для начала задаю состояние для повторяемости эксперимента

In [27]:
state = np.random.RandomState(12345)

Ниже написана функция расчета доходности. На вход подаются целевые (реальные) значения случайно выбранных 500 записей и соответствующие им 500 предсказанных значений, а так же количество скважин. Количество не стал делать константой для ниверсализации функции. На выходе - доход

In [28]:
def revenue(target,predictions,count):
    predictions = predictions.sort_values(ascending=False)[:count]
    top_200 = target[predictions.index]
    product = top_200.sum()
    income = product * INCOME
    profit = income - BUDGET
    return profit

Бутстрап тоже обернул в функцию для более удобного обращения. На вход подаются: валидационная выборка, предсказания, полученые на валидационной выборке и состояние системы. на выходе три параметра - риск (количество скважин с отрицательной доходностью), срдеднее значение доходности и 95% доверительный интервал.

In [29]:
def bootstrap(y_val, y_pred, state):
    values = []
    #conf_interval = []
    for i in range(1000):
        y_val_subsample = y_val.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        y_pred_series = pd.Series(y_pred)
        y_pred_subsample = y_pred_series[y_val_subsample.index]
        prof = revenue(y_val_subsample, y_pred_subsample, 200)
        values.append(prof)
        #prof_interval = pd.Series(prof)
        #conf_interval.append(y_val_subsample.quantile(0.99))
    values = pd.Series(values)
    risk = (values < 0).sum()
    mean_income = values.mean()
    #interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    #conf_interval = pd.Series(conf_interval)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    return risk, mean_income, lower, upper

Привет, расчет интервала поменял в аналогии с тренажером. conf_interval использую для сохранения 1000 результатов интервалов.

Таким образом в самом цикле квантили вообще не ищутся? 

In [30]:
risk, mean_income, lower, upper = bootstrap(y0_val, y0_pred, state)
print("Первый регион")
print("Риск (доля скважин с отрицательным доходом) =", risk / 1000)
print("Среднее по доходности =", mean_income)
print("95% доверителный интервал, нижняя граница =", lower, " верхняя граница = ", upper)

Первый регион
Риск (доля скважин с отрицательным доходом) = 0.02
Среднее по доходности = 600735244.2611653
95% доверителный интервал, нижняя граница = 12948331.135115242  верхняя граница =  1231163605.7914982


In [31]:
risk, mean_income, lower, upper = bootstrap(y1_val, y1_pred, state)
print("Второй регион")
print("Риск (доля скважин с отрицательным доходом) =", risk / 1000)
print("Среднее по доходности =", mean_income)
print("95% доверителный интервал, нижняя граница =", lower, " верхняя граница = ", upper)

Второй регион
Риск (доля скважин с отрицательным доходом) = 0.001
Среднее по доходности = 663958995.2601907
95% доверителный интервал, нижняя граница = 206476361.25177094  верхняя граница =  1191197684.7488434


In [32]:
risk, mean_income, lower, upper = bootstrap(y2_val, y2_pred, state)
print("Третий регион")
print("Риск (доля скважин с отрицательным доходом) =", risk / 1000)
print("Среднее по доходности =", mean_income)
print("95% доверителный интервал, нижняя граница =", lower, " верхняя граница = ", upper)

Третий регион
Риск (доля скважин с отрицательным доходом) = 0.025
Среднее по доходности = 597381047.9005232
95% доверителный интервал, нижняя граница = 1734929.531176098  верхняя граница =  1246217960.1652355


## Выводы
На мой взгляд наиболее перспективным регионом для развития является регион под номером 2 (1 по моему исчислению:). Он является наиболее доходным (657739074.145053), имеет наименьшую долю рисковых скважин (0.004) и наименьщий разброс в доверительном интервале (от 198017183.94341135 до 1206811397.865249). 

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован